In [3]:
import spacy

In [14]:
#Select anyother pipeline according to gpu etc.
!python -m spacy download en_core_web_lg 

     -------------------------------------- 587.7/587.7 MB 3.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


2022-10-14 12:41:16.607958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-14 12:41:16.608571: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import json

with open('../../../data/250_sentences_annotated_data.jsonl', 'r') as data_file:
    data = [json.loads(line) for line in data_file]
  


In [3]:
type(data)

list

In [4]:
data[0]

{'id': 3062,
 'text': "Summary\n\nSummary Companies Tesla down as Q3 deliveries miss market estimates\n\nU.S. factory activity slowest in ~2.5 years in Sept -ISM\n\nCredit Suisse, Citi cut 2022 year-end target for S&P 500\n\nIndexes up: Dow 2.66%, S&P 500 2.59%, Nasdaq 2.27%\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOct 3 (Reuters) - Wall Street's three major indexes rallied to close over 2% on Monday as U.S. Treasury yields tumbled on weaker-than-expected manufacturing data, increasing the appeal of stocks at the start of the year's final quarter.",
 'label': [[27, 33, 'CUSTOM_ORG'],
  [78, 81, 'CUSTOM_PLACE'],
  [136, 149, 'CUSTOM_ORG'],
  [151, 156, 'CUSTOM_ORG'],
  [267, 274, 'CUSTOM_ORG'],
  [350, 353, 'CUSTOM_PLACE']]}

In [5]:
#Remove id column and get the data 
training_data = []
for row in data:
  temp = {}
  temp['text'] = row['text']
  temp['entities'] = row['label']
  training_data.append(temp)
  
print(training_data[0])

{'text': "Summary\n\nSummary Companies Tesla down as Q3 deliveries miss market estimates\n\nU.S. factory activity slowest in ~2.5 years in Sept -ISM\n\nCredit Suisse, Citi cut 2022 year-end target for S&P 500\n\nIndexes up: Dow 2.66%, S&P 500 2.59%, Nasdaq 2.27%\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOct 3 (Reuters) - Wall Street's three major indexes rallied to close over 2% on Monday as U.S. Treasury yields tumbled on weaker-than-expected manufacturing data, increasing the appeal of stocks at the start of the year's final quarter.", 'entities': [[27, 33, 'CUSTOM_ORG'], [78, 81, 'CUSTOM_PLACE'], [136, 149, 'CUSTOM_ORG'], [151, 156, 'CUSTOM_ORG'], [267, 274, 'CUSTOM_ORG'], [350, 353, 'CUSTOM_PLACE']]}


#####  To train spacy model,we have to convert our data to format spacy understands which is Doc

In [6]:
#Create empty spacy model and DocBin
from spacy.tokens import DocBin
from tqdm import tqdm

blank_model = spacy.blank("en") 
doc_bin = DocBin()

In [7]:
# Convert training data
from spacy.util import filter_spans

for sample  in tqdm(training_data): 
    text = sample['text']
    labels = sample['entities']
    doc = blank_model.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skip")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|██████████| 250/250 [00:00<00:00, 1198.07it/s]

Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip
Skip


In [8]:
#Basic config generated from spacy website https://spacy.io/usage/training#quickstart
#Selected ner on UI
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-10-17 21:23:08.184309: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-17 21:23:08.184937: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-17 21:23:11.987705: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-17 21:23:11.989469: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-10-17 21:23:11.991256: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-10-17 21:23:11.993413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [10]:
#start training
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     31.56    0.00    0.00    0.00    0.00
✔ Saved pipeline to output directory
model-last


2022-10-17 21:56:05.867020: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-17 21:56:05.867600: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-17 21:56:10.997164: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-17 21:56:10.999356: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublas64_11.dll'; dlerror: cublas64_11.dll not found
2022-10-17 21:56:11.002041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cublasLt64_11.dll'; dlerror: cublasLt64_11.dll not found
2022-10-17 21:56:11.004669: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cu

In [1]:
import shutil
shutil.move("model-best", "../../models/spacy_model")

'../../models/spacy_model'

In [7]:
# Inference Code
nlp_ner = spacy.load("../../models/spacy_model")

doc = nlp_ner("Joerg Steinbach, the regional economy minister of Brandenburg, where Tesla has its factory near Berlin") 
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)


Joerg Steinbach 0 15 CUSTOM_PERSON
regional economy minister 21 46 CUSTOM_ROLE
Brandenburg 50 61 CUSTOM_ORG
Tesla 69 74 CUSTOM_ORG
Berlin 96 102 CUSTOM_PLACE


In [8]:
spacy.displacy.render(doc, style="ent")

In [9]:
nlp_ner.get_pipe("ner").labels

('CUSTOM_ORG', 'CUSTOM_PERSON', 'CUSTOM_PLACE', 'CUSTOM_ROLE')